# Error Analysis on 8760 profiles (22 profiles)

In [2]:
#importing packages needed for analysis
import os
import numpy as np
import pandas as pd
import math
from pandas import DataFrame

path = os.getcwd()
#print(path)

#this code creates an output directory in the parent director, if one does not exist yet
#Note: this is where all of the output files will be written, since outputs are large this saves space in git
path = os.getcwd()
parent = os.path.dirname(path)
outputs_dir = parent+'\outputs\error_analysis'
if not os.path.exists(outputs_dir):
    os.makedirs(outputs_dir)
print('output files are written out in parent directory: '+outputs_dir)

##UNCOMMENT WHICH PROFILE BEING ANALYZED 
x = 'load'
x2 = 'Load'

#x = 'solar'
#x2 = 'Solar_Gen'

#x = 'wind'
#x2 = 'Wind_Gen'

output files are written out in parent directory: C:\Users\tgoforth\Documents\IPM temporal resolution project\outputs\error_analysis


## General Error Analysis

difference in values 
root mean square for each region
other analyses? 

## Load in data 
## Choose the profile to use by commenting / uncommenting code

In [197]:
############### GENERAL DATA LOAD ###############
# have to manually input CSV file name if not one of the 22 profiles
#x_segments = pd.read_csv('')
#x_hours = pd.read_csv('')
#x_name = ''

############### IPM APPROACH (2 profiles) ###############
# NORM
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_NORM.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_NORM.csv')
#x_name = 'norm'

# TOD before group
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_timeofday.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_timeofday.csv')
#x_name = 'timeofday'

############### SEQUENTIAL APPROACH (7 profiles) ###############
# 2-hr
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_seq_2hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_seq_2hr.csv')
#x_name = 'seq_2hr'

# 4-hr
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_seq_4hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_seq_4hr.csv')
#x_name = 'seq_4hr'

# 8-hr
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_seq_8hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_seq_8hr.csv')
#x_name = 'seq_8hr'

# 12-hr
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_seq_12hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_seq_12hr.csv')
#x_name = 'seq_12hr'

# 24-hr
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_seq_24hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_seq_24hr.csv')
#x_name = 'seq_24hr'

# 48-hr
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_seq_48hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_seq_48hr.csv')
#x_name = 'seq_48hr'

# 120-hr
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_seq_120hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_seq_120hr.csv')
#x_name = 'seq_120hr'

############### ONE/TWO DAY APPROACH (7 profiles) ###############
# One day type, monthly, 24 hrs
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_1daytype_monthly_24hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_1daytype_monthly_24hr.csv')
#x_name = '1daytype_month_24hr'

# One day type, season, 24 hrs
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_1daytype_season_24hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_1daytype_season_24hr.csv')
#x_name = '1daytype_season_24hr'

# Two day type, monthly, 24 hrs
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_2daytype_monthly_24hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_2daytype_monthly_24hr.csv')
#x_name = '2daytype_month_24hr'

# Two day type, season, 24 hrs#
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_2daytype_season_24hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_2daytype_season_24hr.csv')
#x_name = '2daytype_season_24hr'

# Two day type, monthly, 4 hrs
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_2daytype_month_4hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_2daytype_month_4hr.csv')
#x_name = '2daytype_month_4hr'

# Two day type, bimonthly, 4 hrs
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_2daytype_bimonth_4hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_2daytype_bimonth_4hr.csv')
#x_name = '2daytype_bimonth_4hr'

# Two day type, season, 4 hrs
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_2daytype_season_4hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_2daytype_season_4hr.csv')
#x_name = '2daytype_season_4hr'

############### THREE DAY TYPE APPROACH (6 profiles) ###############
# Three day type, monthly, 24 hrs
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_3daytype_monthly_24hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_3daytype_monthly_24hr.csv')
#x_name = '3daytype_month_24hr'

# Three day type, season, 24 hrs
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_3daytype_season_24hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_3daytype_season_24hr.csv')
#x_name = '3daytype_season_24hr'

# Three day type, annual, 24 hrs
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_3daytype_annual_24hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_3daytype_annual_24hr.csv')
#x_name = '3daytype_annual_24hr'

# Three day type, bimonthly, 4 hrs
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_3daytype_bimonth_4hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_3daytype_bimonth_4hr.csv')
#x_name = '3daytype_bimonth_4hr'

# Three day type, season based, 4 hrs
#x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_3daytype_seasonbased_4hr.csv')
#x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_3daytype_seasonbased_4hr.csv')
#x_name = '3daytype_seasonbased_4hr'

# Three day type, season, 4 hrs
x_segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_3daytype_season_4hr.csv')
x_hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_3daytype_season_4hr.csv')
x_name = '3daytype_season_4hr'

## Create outputs: total number of representative hours, RMS

In [198]:
#print(x_hours)

# use groupby region to count number of representative hours in each region
aggregations = {'Avg':'count'}
stat1 = x_segments.groupby(['Region'],as_index=False).agg(aggregations)
stat1 = stat1.rename(columns={'Avg':'Rep_Count'})
#stat1 = stat1[['Region','Count']].copy()

print(stat1)

      Region  Rep_Count
0   ERC_REST         54
1   ERC_WEST         54
2       FRCC         54
3   MIS_AMSO         54
4     MIS_AR         54
..       ...        ...
58   WECC_WY         54
59  WEC_BANC         54
60  WEC_CALN         54
61  WEC_LADW         54
62  WEC_SDGE         54

[63 rows x 2 columns]


In [199]:
# calculate RMSE 
stat2 = x_hours[['Region','R_Group','Hour_Tot',x2,'Avg']].copy()
stat2 = pd.merge(stat2,stat1,on='Region',how='left')
stat2 = stat2[['Region','R_Group','Rep_Count','Hour_Tot',x2,'Avg']]

#print(stat2)

stat2['Diff'] = stat2[x2] - stat2['Avg'] 
stat2['Square'] = stat2['Diff']**2
stat_reg = stat2.groupby(['Region','Avg','Hour_Tot'],as_index=False).agg({'Square' : sum})
stat_reg = stat_reg.rename(columns = {'Square':'Square_Sum'}) 
stat_reg['Mean'] = stat_reg['Square_Sum'] / stat_reg['Hour_Tot'] 
stat_reg['RMSE'] = stat_reg['Mean']**(1/2)
print(stat_reg)

stat2 = pd.merge(stat2,stat_reg,on=['Region','Avg','Hour_Tot'],how='left')
print(stat2)

stat2.to_csv('../outputs/error_analysis/'+x+'_'+x_name+'_error.csv')
stat_reg.to_csv('../outputs/error_analysis/'+x+'_'+x_name+'region_error.csv')

# RMSE tells us how spread out the data is over a line of best fit

        Region           Avg  Hour_Tot    Square_Sum          Mean  \
0     ERC_REST  29443.558559       111  4.067633e+08  3.664535e+06   
1     ERC_REST  29796.642659       361  1.622126e+09  4.493425e+06   
2     ERC_REST  32400.546218       119  1.664162e+09  1.398456e+07   
3     ERC_REST  32597.629139       151  1.629911e+09  1.079411e+07   
4     ERC_REST  33159.873362       229  3.582208e+09  1.564283e+07   
...        ...           ...       ...           ...           ...   
3397  WEC_SDGE   3093.250000        16  1.305863e+06  8.161644e+04   
3398  WEC_SDGE   3145.583333        12  4.659269e+05  3.882724e+04   
3399  WEC_SDGE   3184.750000        20  3.922880e+06  1.961440e+05   
3400  WEC_SDGE   3582.200000        20  5.263615e+06  2.631808e+05   
3401  WEC_SDGE   3701.300000        20  5.218936e+06  2.609468e+05   

             RMSE  
0     1914.297419  
1     2119.770006  
2     3739.593398  
3     3285.439791  
4     3955.101522  
...           ...  
3397   285.685907  

In [200]:
# descriptive statistics of entire data set 
stat2['Diff'].describe()

count    5.518800e+05
mean    -4.323028e-12
std      1.277079e+03
min     -1.923308e+04
25%     -2.518426e+02
50%     -5.064452e+00
75%      2.379612e+02
max      1.975285e+04
Name: Diff, dtype: float64

In [201]:
# descriptive statistics of only a single region
x_FRCC = stat2[stat2['R_Group']=='FRCC']
print('number of rows in dataset for testing =',x_FRCC.shape[0])

x_FRCC_stat = x_FRCC['Diff'].describe()
print(x_FRCC_stat)


## TO DO: create DF of all stats for all 22 profiles ###
x_FRCC_stat.to_csv('../outputs/error_analysis/'+x+'_'+x_name+'_FRCC_stat.csv')

number of rows in dataset for testing = 8760
count    8.760000e+03
mean    -2.493511e-11
std      3.048958e+03
min     -1.143375e+04
25%     -1.917797e+03
50%     -1.611372e+02
75%      1.803554e+03
max      1.975285e+04
Name: Diff, dtype: float64
